# Revised case normalization for KSSG Spitalregion Rheintal, Werdenberg, Sarganserland 2019

Notes:

- 47 of 48 revised DtoD cases were joined with the BFS cases from the DB.



In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/Linth_Toggenburg_SRRWS_2019.xlsx'),
         'KSSG Spitalregion Rheintal, Werdenberg, Sarganserland',
         '2019',
         'Änderungen SRRWS_2019')

print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Linth_Toggenburg_SRRWS_2019.xlsx', hospital_name_db='KSSG Spitalregion Rheintal, Werdenberg, Sarganserland', year='2019', sheet='Änderungen SRRWS_2019')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-21 09:22:57.489 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:38 - Read 48 cases for KSSG Spitalregion Rheintal, Werdenberg, Sarganserland 2019
2022-11-21 09:22:57.494 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:73 - TYPES:
tranche                   string
case_id                   object
patient_id                object
kkik                      string
gender                    string
age_years                  int64
bfs_code                  string
duration_of_stay           int64
pflegetage neu            string
pccl                       int64
pccl neu                  string
old_pd                    string
primary_diagnosis         string
added_icds                string
removed_icds              string
added_chops               string
removed_chops             string
drg                       string
drg neu                   string
cw alt                    string
cw neu                    string

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,2010216467,971401A627916522,W,67,16,2010216467,M7285,M7285,M200,2,I27D,"[D62, L0311]",[D648],[],[]
1,2010230894,0FD7D457C22B4ECD,M,75,10,2010230894,S510,S510,M100,3,X06B,[R18],[],[],[]
2,2010233003,D6D96CE0A68739FB,W,70,14,2010233003,K432,K432,M200,3,G21B,[D62],[D649],[],[]
3,2010234179,FB8869E03D85B3F5,M,59,7,2010234179,K868,K868,M100,3,H62B,[I5014],[I509],[],[]
4,2070263179,A8239CB0553D4656,W,79,8,2070263179,I441,I441,M100,3,F12D,[J9580],[],[],[]
5,2010220228,0684D4B0A48A69C8,M,60,14,2010220228,D120,D120,M200,3,G18B,"[K650, K358]",[],[],[]
6,2010228707,51AA6AD90E4E760A,W,58,8,2010228707,K5722,K5722,M200,3,G18B,[D6988],[],[],[]
7,2010229560,D070499746D5BD08,M,52,11,2010229560,K500,K500,M200,3,G18B,[D6988],[],[],[]
8,2010230678,0B867D70DE4BD6E0,M,77,18,2010230678,D122,D122,M200,3,G18B,"[K913, D6988]",[],[990A],[]
9,2070282416,C59CE74D4CD08931,W,77,8,2070282416,K565,K565,M200,3,G18B,[S3681],[S3640],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

2022-11-21 09:23:06.039 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:95 - Read 14990 rows from the DB, for the hospital 'KSSG Spitalregion Rheintal, Werdenberg, Sarganserland' in 2019
2022-11-21 09:23:06.051 | WARNING  | src.revised_case_normalization.notebook_functions.revise:revise:51 - 1 rows could not be matched, given ['age_years', 'case_id_norm', 'duration_of_stay', 'gender']


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,588648,2010229750,R55,"[S2206, X599, B022, I1000, N390, ...]",,[],W,90,0,0,6,0,01,2019-07-04,0,00,2019-07-10
1,589124,2070267501,A419,"[N390, J101, E049, E876, E870, ...]",998425::20190227,[99B813::20190227],W,66,0,0,12,0,01,2019-02-27,0,00,2019-03-11
2,589375,2010231388,C670,"[C672, D62, Z921, I7021, Z867, ...]",574912::20190805,"[990410::20190730, 99B711::20190730, 570X12::20190806, 009910::20190806]",M,66,0,0,9,0,01,2019-07-30,0,00,2019-08-08
3,589425,2010231467,K262,"[K227, D62, N183, N1799, E871, ...]",444399::20190802,"[990410::20190803, 99B713::20190802]",M,81,0,0,10,0,01,2019-08-02,0,00,2019-08-12
4,590408,2010237009,K8580,"[K860, K862, F102, I1000, R600, ...]",,[],M,55,0,0,8,0,01,2019-10-17,0,06,2019-10-25


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops,aimedic_id,case_id_db,patient_id_db,age_days,admission_weight,gestation_age,admission_date,grouper_admission_type,discharge_date,grouper_discharge_type,ventilation_hours
28,2070265027,2884A07656CD9535,M,44,13,2070265027,S321,S321,M200,3,I68B,"[S334, B181]",[],[846],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-21 09:23:06.423 | INFO     | src.revised_case_normalization.notebook_functions.group:group:86 - Grouping 47 cases ...
2022-11-21 09:23:07.402 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:91 - Grouped 47 cases into: 47 revisions, 486 diagnoses rows, 126 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,588648,F73A,1.01,1.01,4,2022-12-31
1,589124,T60D,1.42,1.42,4,2022-12-31
2,589375,L18A,1.76,1.76,4,2022-12-31
3,589425,G46A,1.88,1.88,4,2022-12-31
4,590408,H62A,1.91,1.13,4,2022-12-31
5,591027,G04Z,3.33,3.33,4,2022-12-31
6,591032,G67A,1.71,1.71,4,2022-12-31
7,592184,N06A,1.53,1.53,3,2022-12-31
8,592320,G13Z,4.12,4.12,4,2022-12-31
9,592429,I27A,2.89,2.89,4,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-21 09:23:07.422 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:259 - Trying to insert 47 cases into the 'Revisions' table ...
2022-11-21 09:23:08.387 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:285 - Deleted 47 rows from the "Revisions" table, which is about to be updated
2022-11-21 09:23:08.460 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:296 - Inserted 47 cases into the 'Revisions' table
2022-11-21 09:23:08.464 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:309 - Trying to insert 486 rows into the 'Diagnoses' table ...
2022-11-21 09:23:10.299 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:347 - Inserted 486 rows into the 'Diagnoses' table
2022-11-21 09:23:10.300 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:358 - Trying to insert 126 rows into the 'Procedures' ta